# Fantasy Premier League Moneyball

#### Optimisation constraints:
- My budget (start at £100 million but can go up and down based on player prices changing)
- Fill a team of 15 players consisting of 
        - 2 x GKP
        - 5 x DEF
        - 5 x MID
        - 3 X FWD
- 4 of these players will be subs and do not generate you gamepoints.
- The 'starting 11' Can be of any combination/formation, providing 1 goalkeeper, at least 3 defenders and at least 1 forward are selected at all times.
- Only 1 free transfer can be made each week, further transfers deduct 4 points from your total points


#### We want to Maximise:
- Total points per player
- Player score (points per million x fixture multiplier) x chance to play


#### Caps
- You are given a selection of boosts / advantages (called Caps) which you can play at any time, these are:
        - 2 x Wildcards: Make unlimited free transfers to your team in a gameweek
        - 1 x Free Hit: Same as above, but team reverts back to previous lineup after 1 week
        - Triple Captain: Your captain scores you triple points for the next gameweek
        - Bench Boost: The points from all your subs are included in your total score for the gameweek




In [ ]:
# The plan:

# link to google sheet:
# https://docs.google.com/spreadsheets/d/1z-DRmtGm3pW-xp951SmK0VTQidB_CxWMNmmciaQQc8M/edit#gid=581306190

# 1. Pull data from fantasy premier league site to create datasets for:
    # Club data - DONE
    # Player data - DONE
    # Fixture data - DONE - add in the fixture multipliers here
    # My team data - DONE - need to add player score once calculated
    # Previous season player data - NEED TO CREATE DF FROM CSV :
        #'C:\Users\andre\Documents\GitHub\murry_code\Fantasy Premier League\FPL_Players-2019-2020 season'
    
# 2. Create player score based on:
    # A - Points per million this season - DONE
    # B - Points per million last season (proportionate to games played by team so far this season, with a lower weighting)
    #   - Requires appearances to be calculated 

    # C - Blended Nxt 3 Gameweek Multiplier (average(next gw multiplier, avergage(next 3 gw multiplier))
    
    # D - Chances of playing next round multiplier (excludes injury & suspended) - DONE 
    
    # Player score = ((A + B) x C)) x D
    
# 3. Create Optimal team:    
    # Create best team of: 
        # Using 80-82 mil
        # 1 x GK
        # 3 x DEF
        # 4 x MID
        # 3 X FOR

    # Leaving enough for Subs: 
        # 18-20mil
        # 1 x GK
        # 2 x DEF
        # 1 X MID

# 4. Figure out a way to make transfer most efficiently to maximise my team score, 
    # Considering 1 free transfer, then 4 points per transfer after that       
    # calculate points predicted to score in next gameweek of lowest in active team 
    # vs 
    # position vs highest points predicted of player not in team
    
# Additional ideas
    # Pull data from promoted teams from the EFL championship in previous year so there is data on newly promoted teams/players
    # Add logic of points per appearance to get around lower points for injured or booked players
    # Possibly mins per appearance to infer a likliness to play
    # Add in a 'strongest fixture of the season' flag for players for when to play triple captain
    # Add in 'strongest fixture of next gameweek' flag for current capatin / vice captain
    

### Creating variables for the api urls and logging in using personal details

In [ ]:
#############
## imports ##
#############

import pandas as pd
from datetime import datetime as dt
import requests
import numpy as np


###########################
## URLs an Login Details ##
###########################

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 100

# Code for my team picks
my_team_id = '3366741'

# Current Datetime for csv outputs
datetime = dt.now().strftime("%Y%m%d-%H%M%S")

current_gameweek = 2
next_five_gameweek = range(current_gameweek, current_gameweek+5)

## URLs

## login url to create a session using my login details ## 
login_url = 'https://users.premierleague.com/accounts/login/'

## api url for the main dataset on player stats 
main_url = 'https://fantasy.premierleague.com/api/bootstrap-static/'

## api for my team
team_url = 'https://fantasy.premierleague.com/api/my-team/3366741/'

## api for god knows what??
entry_url = 'https://fantasy.premierleague.com/api/entry/3366741/'

## api for fixtures
fix_url = 'https://fantasy.premierleague.com/api/fixtures/'

session = requests.session()

## Create session with login details for urls which require login ##
## Add pasword and login email
payload = {
 'password': '',
 'login': '',
 'redirect_uri': 'https://fantasy.premierleague.com/a/login',
 'app': 'plfpl-web'
}
session.post(login_url, data=payload)


### Creating data frames and csv outputs

In [ ]:
## CLUB DATAFRAME AND CSV EXTRACT ##
### Creating a dataframe for the club data and saving as csv ###

############################
#### dataframe creation ####
############################

## Converting the data from the url to json then pulling out the teams data to a separate 'club' data frame
main_req = requests.get(main_url)
main_json = main_req.json()
main_json.keys()
club_df = pd.DataFrame(main_json['teams'])

# print(club_df.head())
club_df.head()

#####################
##### csv output ####
#####################

## Specifying the filename and path (Location to export the CSV to)
# club_filename = 'FPL_Clubs-'
# datetime = dt.now().strftime("%Y%m%d-%H%M%S")
# club_path = 'C:\\Python CSV Output\\'+club_filename+datetime+'.csv'

## Saved dataframe as csv
# club_df.to_csv(club_path, index = False)

In [ ]:
### PREVIOUS SEASON PLAYER DATA ###
### Creating a dataframe for the player data and saving as csv ###
### Several uses, will contain the player score used to determine best team / transfers ### 

############################
#### Dataframe Creation ####
############################

import_previous_season_path = r'C:\Users\andrew.morris\Documents\GitHub\murry_code\Fantasy Premier League\\'
prev_season_filename = "FPL_Players_2019_2020_season_fudged.csv"
prev_season_player_df = pd.read_csv(import_previous_season_path+prev_season_filename) 
previous_season_games = 38

prev_season_player_df['points_per_gameweek'] = round(prev_season_player_df.total_points / previous_season_games,2)
    
# print(prev_season_player_df.head())    
                                                                                     

In [ ]:
### PLAYER DATA ###
### Creating a dataframe for the player data and saving as csv ###
### Several uses, will contain the player score used to determine best team / transfers ### 

############################
#### Dataframe Creation ####
############################

full_player_df = pd.DataFrame(main_json['elements'])         # Player dataframe
position_types_df = pd.DataFrame(main_json['element_types']) # Position dataframe
events_df = pd.DataFrame(main_json['events'])                # Gameweek dataframe

# Updating and creating some of the fields in full player dataframe: 
    # Pulling through position and team name instead of 'id', and changing data types
full_player_df['position'] = full_player_df.element_type.map(position_types_df.set_index('id').singular_name)
full_player_df['position_abv'] = full_player_df.element_type.map(position_types_df.set_index('id').singular_name_short)
full_player_df['team'] = full_player_df.team.map(club_df.set_index('id').name)
full_player_df['team_abv'] = full_player_df.team.map(club_df.set_index('name').short_name)

    # Pulling through data related to previous season
full_player_df['prev_season_points'] = full_player_df.id.map(prev_season_player_df.set_index('id').total_points).fillna(0.0)
full_player_df['prev_season_points_per_gw'] = full_player_df.id.map(prev_season_player_df.set_index('id').points_per_gameweek).fillna(0.0)
full_player_df['prev_season_minutes'] = full_player_df.id.map(prev_season_player_df.set_index('id').minutes).fillna(0.0)

    # Converting selected_by_percent to float data type
full_player_df['selected_by_percent'] = full_player_df.selected_by_percent.astype(float)

    # Creating chance of playing next round multiplier
full_player_df['next_round_play_chance'] = full_player_df['chance_of_playing_next_round']/100.0
full_player_df['next_round_play_chance'] = full_player_df.next_round_play_chance.fillna(1.0)
full_player_df['this_round_play_chance'] = full_player_df['chance_of_playing_this_round']/100.0
full_player_df['this_round_play_chance'] = full_player_df.this_round_play_chance.fillna(1.0)

    # Creating custom points_per_mil
full_player_df['cost_in_mil'] = full_player_df['now_cost']/10
full_player_df['points_per_mil'] = round(full_player_df['total_points']/full_player_df['cost_in_mil'],2)
full_player_df['prev_season_points_per_mil'] = ((round(full_player_df['prev_season_points']/full_player_df['cost_in_mil'],2))/38)*(current_gameweek-1)

full_player_df['current_gameweek'] = current_gameweek

# print(full_player_df)

# Creating a slimline version of the elements table with relevant data points required for players
slim_player_df = full_player_df[['first_name',
                                 'second_name',
                                 'web_name',   
                                 'id',   
                                 'position',
                                 'position_abv',
                                 'team',
                                 'team_abv',
                                 'selected_by_percent', 
                                 'current_gameweek',
                                 'this_round_play_chance',   
                                 'next_round_play_chance',      
                                 'status',
                                 'cost_in_mil',
                                 'minutes',
                                 'transfers_in',
                                 'transfers_out',   
                                 'total_points',
                                 'prev_season_minutes',
                                 'prev_season_points',
                                 'prev_season_points_per_gw',
                                 'prev_season_points_per_mil',
                                 'points_per_mil']]


# print(slim_player_df.head(10))

#####################
##### csv output ####
#####################

## Specifying the filename and path (Location to export the CSV to)
player_filename = 'FPL_Players-'
datetime = dt.now().strftime("%Y%m%d-%H%M%S")
player_path = 'C:\\Python CSV Output\\'+player_filename+datetime+'.csv'

# ## Exporting dataframe as csv
# slim_player_df.to_csv(player_path, index = False, encoding="utf8")

slim_player_df.head()

In [ ]:
DEF_home_fix_df.shape

In [ ]:
### FIXTURE DATA ###
### Creating a dataframe for the fixtures ###
### Primarily used to create a gameweek difficulty multiplier which is applied to the player score

#########################################
#### full fixture dataframe creation ####
#########################################

## Getting fixture data from the api url
fix_req = requests.get(fix_url)

## Converting that data to json then dataframe
fix_json = fix_req.json()
fix_df = pd.DataFrame(fix_json)

# Replacing null game weeks with 99 and converting data type to int, required for matches not yet scheduled
fix_df['event'] = fix_df.event.fillna(99.0).astype(int)

# Adding club names and abbreviations as new columns
fix_df['home_team_full']  = fix_df.team_h.map(club_df.set_index('id').name)
fix_df['home_team_short'] = fix_df.team_h.map(club_df.set_index('id').short_name)
fix_df['away_team_full']  = fix_df.team_a.map(club_df.set_index('id').name)
fix_df['away_team_short'] = fix_df.team_a.map(club_df.set_index('id').short_name)

# Adding Home/Away Strengths to fix_df then blending the home and away strengths, 
# As there are no crowds this year the home advantage might be underplayed
# Logic which utilises the home and away scores separately is saved under random at the end of the code

# Keep an eye on this to see if they flip around at any point
    # Home team strength scores
fix_df['h_team_str_overall_home'] = fix_df.team_h.map(club_df.set_index('id').strength_overall_away)
fix_df['h_team_str_att_home'] = fix_df.team_h.map(club_df.set_index('id').strength_attack_away)
fix_df['h_team_str_def_home'] = fix_df.team_h.map(club_df.set_index('id').strength_defence_away)
fix_df['h_team_str_overall_away'] = fix_df.team_h.map(club_df.set_index('id').strength_overall_home)
fix_df['h_team_str_att_away'] = fix_df.team_h.map(club_df.set_index('id').strength_attack_home)
fix_df['h_team_str_def_away'] = fix_df.team_h.map(club_df.set_index('id').strength_defence_home)

    # Blending home and away scores for the home team
fix_df['h_team_str_overall']  = fix_df[['h_team_str_overall_home','h_team_str_overall_away']].mean(axis=1)
fix_df['h_team_str_att'] = fix_df[['h_team_str_att_home','h_team_str_att_away']].mean(axis=1)
fix_df['h_team_str_def'] = fix_df[['h_team_str_def_home','h_team_str_def_away']].mean(axis=1)

    # Away team strength scores
fix_df['a_team_str_overall_away'] = fix_df.team_a.map(club_df.set_index('id').strength_overall_home)
fix_df['a_team_str_att_away'] = fix_df.team_a.map(club_df.set_index('id').strength_attack_home)
fix_df['a_team_str_def_away'] = fix_df.team_a.map(club_df.set_index('id').strength_defence_home)
fix_df['a_team_str_overall_home'] = fix_df.team_a.map(club_df.set_index('id').strength_overall_away)
fix_df['a_team_str_att_home'] = fix_df.team_a.map(club_df.set_index('id').strength_attack_away)
fix_df['a_team_str_def_home'] = fix_df.team_a.map(club_df.set_index('id').strength_defence_away)
    
    # Blending home and away scores for the away team
fix_df['a_team_str_overall']  = fix_df[['a_team_str_overall_home','a_team_str_overall_away']].mean(axis=1)
fix_df['a_team_str_att'] = fix_df[['a_team_str_att_home','a_team_str_att_away']].mean(axis=1)
fix_df['a_team_str_def'] = fix_df[['a_team_str_def_home','a_team_str_def_away']].mean(axis=1)


# Creating home/away position specific multipliers
    # Basic logic is:
        # For goalkeepers or defenders - home team defense score / away team attack score
        # For midfielders - home team overall score / away team overall score
        # For forwards - home team attack score / away team defense score
    
    # Home team multipliers
fix_df['home_GKP_DEF_multi'] = fix_df['h_team_str_def'] / fix_df['a_team_str_att']
fix_df['home_MID_multi'] = fix_df['h_team_str_overall'] / fix_df['a_team_str_overall']
fix_df['home_FWD_multi'] = fix_df['h_team_str_att'] / fix_df['a_team_str_def']

    # Away team multiplier
fix_df['away_GKP_DEF_multi'] = fix_df['a_team_str_def'] / fix_df['h_team_str_att']
fix_df['away_MID_multi'] = fix_df['a_team_str_overall'] / fix_df['h_team_str_overall']
fix_df['away_FWD_multi'] = fix_df['a_team_str_att'] / fix_df['h_team_str_def']

# Renaming 'event' to 'gameweek'
fix_df = fix_df.rename(columns={"event": "gameweek"})

# print(fix_df)

#######################################################
### SLIM FIXTURES DATAFRAME AND GAMEWEEK MULTIPLIER ###
#######################################################


# Creating a slim version of the fixtures dataframe with a row per position, per team, per fixture  
# Firstly done by creating a dataframe for each position for home and away teams then concatenating all the dataframes

###################################
### Goalkeepers playing at home ###
###################################

GKP_home_fix_df = fix_df[['gameweek',
                          'kickoff_time',
                          'team_h',
                          'home_team_full',
                          'home_team_short',
                          'away_team_full',
                          'away_team_short',
                          'team_h_difficulty',
                          'home_GKP_DEF_multi'
                     ]]

GKP_home_fix_df['position_abv'],GKP_home_fix_df['home_or_away'] = ['GKP','home']

GKP_home_fix_df = GKP_home_fix_df.rename(columns={'team_h':'team_id',
                                                  'home_team_full':'team',
                                                  'home_team_short':'team_abv',
                                                  'away_team_full':'opponent',
                                                  'away_team_short':'opponent_abv',
                                                  'team_h_difficulty':'match_difficulty',
                                                  'home_GKP_DEF_multi':'match_multiplier'
                                                 })

#################################
### Defenders playing at home ###
#################################

DEF_home_fix_df = fix_df[['gameweek',
                          'kickoff_time',
                          'team_h',
                          'home_team_full',
                          'home_team_short',
                          'away_team_full',
                          'away_team_short',
                          'team_h_difficulty',
                          'home_GKP_DEF_multi'
                         ]]

DEF_home_fix_df['position_abv'],DEF_home_fix_df['home_or_away'] = ['DEF','home']

DEF_home_fix_df = DEF_home_fix_df.rename(columns={'team_h':'team_id',
                                                  'home_team_full':'team',
                                                  'home_team_short':'team_abv',
                                                  'away_team_full':'opponent',
                                                  'away_team_short':'opponent_abv',
                                                  'team_h_difficulty':'match_difficulty',
                                                  'home_GKP_DEF_multi':'match_multiplier'
                                                 })

###################################
### Midfielders playing at home ###
###################################

MID_home_fix_df = fix_df[['gameweek',
                          'kickoff_time',
                          'team_h',
                          'home_team_full',
                          'home_team_short',
                          'away_team_full',
                          'away_team_short',
                          'team_h_difficulty',
                          'home_MID_multi',
                         ]]

MID_home_fix_df['position_abv'],MID_home_fix_df['home_or_away'] = ['MID','home']

MID_home_fix_df = MID_home_fix_df.rename(columns={'team_h':'team_id',
                                                  'home_team_full':'team',
                                                  'home_team_short':'team_abv',
                                                  'away_team_full':'opponent',
                                                  'away_team_short':'opponent_abv',
                                                  'team_h_difficulty':'match_difficulty',
                                                  'home_MID_multi':'match_multiplier',
                                                 })

################################
### Forwards playing at home ###
################################

FWD_home_fix_df = fix_df[['gameweek',
                          'kickoff_time',
                          'team_h',
                          'home_team_full',
                          'home_team_short',
                          'away_team_full',
                          'away_team_short',
                          'team_h_difficulty',
                          'home_FWD_multi'
                         ]]

FWD_home_fix_df['position_abv'],FWD_home_fix_df['home_or_away'] = ['FWD','home']

FWD_home_fix_df = FWD_home_fix_df.rename(columns={'team_h':'team_id',
                                                  'home_team_full':'team',
                                                  'home_team_short':'team_abv',
                                                  'away_team_full':'opponent',
                                                  'away_team_short':'opponent_abv',
                                                  'team_h_difficulty':'match_difficulty',
                                                  'home_FWD_multi':'match_multiplier'
                                                 })

################################
### Goalkeepers playing away ###
################################

GKP_away_fix_df = fix_df[['gameweek',
                          'kickoff_time',
                          'team_a',
                          'away_team_full',
                          'away_team_short',
                          'home_team_full',
                          'home_team_short',
                          'team_a_difficulty',
                          'away_GKP_DEF_multi'
                         ]]

GKP_away_fix_df['position_abv'],GKP_away_fix_df['home_or_away'] = ['GKP','home']

GKP_away_fix_df = GKP_away_fix_df.rename(columns={'team_a':'team_id',
                                                  'away_team_full':'team',
                                                  'away_team_short':'team_abv',
                                                  'home_team_full':'opponent',
                                                  'home_team_short':'opponent_abv',
                                                  'team_a_difficulty':'match_difficulty',
                                                  'away_GKP_DEF_multi':'match_multiplier'
                                                 })

##############################
### Defenders playing away ###
##############################

DEF_away_fix_df = fix_df[['gameweek',
                          'kickoff_time',
                          'team_a',
                          'away_team_full',
                          'away_team_short',
                          'home_team_full',
                          'home_team_short',
                          'team_a_difficulty',
                          'away_GKP_DEF_multi'
                         ]]

DEF_away_fix_df['position_abv'],DEF_away_fix_df['home_or_away'] = ['DEF','home']

DEF_away_fix_df = DEF_away_fix_df.rename(columns={'team_a':'team_id',
                                                  'away_team_full':'team',
                                                  'away_team_short':'team_abv',
                                                  'home_team_full':'opponent',
                                                  'home_team_short':'opponent_abv',
                                                  'team_a_difficulty':'match_difficulty',
                                                  'away_GKP_DEF_multi':'match_multiplier'
                                                 })

################################
### Midfielders playing away ###
################################

MID_away_fix_df = fix_df[['gameweek',
                          'kickoff_time',
                          'team_a',
                          'away_team_full',
                          'away_team_short',
                          'home_team_full',
                          'home_team_short',
                          'team_a_difficulty',
                          'away_MID_multi'
                         ]]

MID_away_fix_df['position_abv'],MID_away_fix_df['home_or_away'] = ['MID','home']

MID_away_fix_df = MID_away_fix_df.rename(columns={'team_a':'team_id',
                                                  'away_team_full':'team',
                                                  'away_team_short':'team_abv',
                                                  'home_team_full':'opponent',
                                                  'home_team_short':'opponent_abv',
                                                  'team_a_difficulty':'match_difficulty',
                                                  'away_MID_multi':'match_multiplier'
                                                 })

#############################
### Forwards playing away ###
#############################

FWD_away_fix_df = fix_df[['gameweek',
                          'kickoff_time',
                          'team_a',
                          'away_team_full',
                          'away_team_short',
                          'home_team_full',
                          'home_team_short',
                          'team_a_difficulty',
                          'away_FWD_multi'
                         ]]

FWD_away_fix_df['position_abv'],FWD_away_fix_df['home_or_away'] = ['FWD','home']

FWD_away_fix_df = FWD_away_fix_df.rename(columns={'team_a':'team_id',
                                                  'away_team_full':'team',
                                                  'away_team_short':'team_abv',
                                                  'home_team_full':'opponent',
                                                  'home_team_short':'opponent_abv',
                                                  'team_a_difficulty':'match_difficulty',
                                                  'away_FWD_multi':'match_multiplier'
                                                 })

## Combining all the above dataframes

slim_fix_df = pd.concat([GKP_home_fix_df,
                         DEF_home_fix_df,
                         MID_home_fix_df,
                         FWD_home_fix_df,
                         GKP_away_fix_df,
                         DEF_away_fix_df,
                         MID_away_fix_df,
                         FWD_away_fix_df
                        ])

# Adding boolean fields for identifying current, historic, future and next 5 games weeks 
slim_fix_df['is_current_gameweek'   ] = slim_fix_df.gameweek == current_gameweek 
slim_fix_df['is_historic_gameweek'  ] = slim_fix_df.gameweek <  current_gameweek
slim_fix_df['is_future_gameweek'    ] = slim_fix_df.gameweek >  current_gameweek
slim_fix_df['is_in_next_3_gameweeks'] = slim_fix_df['gameweek'].apply(lambda a: a in range(current_gameweek, current_gameweek+3))
slim_fix_df['is_in_next_5_gameweeks'] = slim_fix_df['gameweek'].apply(lambda a: a in range(current_gameweek, current_gameweek+5))

# Concatenating team_abv and position_abv to use for mapping 
slim_fix_df['team_and_position'] = slim_fix_df.team_abv + '-' + slim_fix_df.position_abv

# Sorting by kickoff datetime 
slim_fix_df = slim_fix_df.sort_values(by=['kickoff_time']).reset_index(drop = True)

print(slim_fix_df.head())

#####################
##### csv output ####
#####################

## Specifying the filename and path (Location to export the CSV to)
fixtures_filename = 'FPL_Fixtures-'
datetime = dt.now().strftime("%Y%m%d-%H%M%S")
fixture_path = 'C:\\Python CSV Output\\'+fixtures_filename+datetime+'.csv'

## Exporting dataframe as csv
# slim_fix_df.to_csv(fixture_path, index = False)

slim_fix_df.head()

In [ ]:
##############################################
### Creating Gameweek Multiplier Dataframe ###
##############################################

# ##########################################
# ### Next 5 Gameweek Average Multiplier ###
# ##########################################

# # Creating a dataframe of just the next 5 gameweeks with columns 
# next_5_gameweeks_multi_df = slim_fix_df[slim_fix_df.is_in_next_5_gameweeks == True]

# # Creating a Pivot table of next 5 gameweeks with 'team_and_position' & average 'match_multiplier' 
# next_5_gameweeks_multi_pivot = next_5_gameweeks_multi_df.pivot_table(index=['team_and_position'],
#                                                           values=['match_multiplier'], 
#                                                           aggfunc=np.mean)                                                  
# # Transforming the pivot table back into a dataframe
# next_5_gameweeks_multi_df = pd.DataFrame(next_5_gameweeks_multi_pivot.to_records()).rename(columns={"match_multiplier": "nxt_5_gw_avg_multi"})

# print(next_5_gameweeks_multi_df.head())

##########################################
### Next 3 Gameweek Average Multiplier ###
##########################################

# Creating a dataframe of just the next 3 gameweeks with columns 
next_3_gameweeks_multi_df = slim_fix_df[slim_fix_df.is_in_next_3_gameweeks == True]

# Creating a Pivot table of next 5 gameweeks with 'team_and_position' & average 'match_multiplier' 
next_3_gameweeks_multi_pivot = next_3_gameweeks_multi_df.pivot_table(index=['team_and_position'],
                                                          values=['match_multiplier'], 
                                                          aggfunc=np.mean)                                                  
# Transforming the pivot table back into a dataframe
next_3_gameweeks_multi_df = pd.DataFrame(next_3_gameweeks_multi_pivot.to_records()).rename(columns={"match_multiplier": "nxt_3_gw_avg_multi"})

print(next_3_gameweeks_multi_df.head())

####################################

# Creating a dataframe of just the next gameweek with columns 'team_and_position' & 'match_multiplier'
gameweek_multiplier_df = slim_fix_df[slim_fix_df.is_current_gameweek == True]
gameweek_multiplier_df = gameweek_multiplier_df[['team_and_position', 'match_multiplier']]

# Renaming match multipiler to 'current_gw_multiplier'
gameweek_multiplier_df = gameweek_multiplier_df.rename(columns={"match_multiplier": "gw_multi"})

# Pulling the 5 week average match multiplier into 'gameweek_multiplier_df' 
# gameweek_multiplier_df['nxt_5_gw_multi'] = gameweek_multiplier_df.team_and_position.map(next_5_gameweeks_multi_df.set_index('team_and_position').nxt_5_gw_avg_multi)
gameweek_multiplier_df['nxt_3_gw_multi'] = gameweek_multiplier_df.team_and_position.map(next_3_gameweeks_multi_df.set_index('team_and_position').nxt_3_gw_avg_multi)

# Taking an average of of the current gameweek multiplier and the next 5 week match multiplier
# This way match strength will still be a little weighted towards the next match
# However the upcoming matches are also taken into account to avoid have to transfer players out of the team again in upcoming weeks
# gameweek_multiplier_df['nxt_5_wk_blended_match_multi'] = gameweek_multiplier_df[['gw_multi','nxt_5_gw_multi']].mean(axis=1)
gameweek_multiplier_df['nxt_3_wk_blended_match_multi'] = gameweek_multiplier_df[['gw_multi','nxt_3_gw_multi']].mean(axis=1)

gameweek_multiplier_df.head()






In [ ]:
full_player_df.dtypes

In [ ]:
### MY TEAM PICKS DATA ###
### Creating a dataframe for my team picks data and saving as csv ###
### Used to see value of current team and highlight:
                                                    # Players to use as captain / subs   
                                                    # When to use wildcards
                                                    # Weak players to transfer out
            
            
session = requests.session()

## Create session with login details for urls which require login ##
payload = {
 'password': '',
 'login': '',
 'redirect_uri': 'https://fantasy.premierleague.com/a/login',
 'app': 'plfpl-web'
}
session.post(login_url, data=payload)            

############################
#### dataframe creation ####
############################

## Getting my team data from the api url, using session as login details required
my_team_ses = (session.get('https://fantasy.premierleague.com/api/my-team/3366741'))

## Converting that data to json then dataframe
my_team_json = my_team_ses.json()

my_team_picks_df = pd.DataFrame(my_team_json['picks'])

# Pulling through player data from 'full_player_df'
my_team_picks_df['cost_in_mil'] = my_team_picks_df.selling_price / 10  

my_team_picks_df = my_team_picks_df.merge(full_player_df[['id',
                                                           'first_name',
                                                           'second_name',
                                                           'web_name',
                                                           'total_points',
                                                           'points_per_mil',
                                                           'position_abv',
                                                           'team_abv',
                                                           'prev_season_points_per_gw',
                                                           'next_round_play_chance'
                                                          ]], 
                                           how = 'left', left_on = ['element'], right_on = ['id'])

# my_team_picks_df['first_name'] = my_team_picks_df.element.map(full_player_df.set_index('id').first_name)
# my_team_picks_df['second_name'] = my_team_picks_df.element.map(full_player_df.set_index('id').second_name)
# my_team_picks_df['web_name'] = my_team_picks_df.element.map(full_player_df.set_index('id').web_name)
my_team_picks_df['potential_profit'] = my_team_picks_df.purchase_price - my_team_picks_df.purchase_price
# my_team_picks_df['total_points'] = my_team_picks_df.element.map(full_player_df.set_index('id').total_points)
# my_team_picks_df['points_per_mil'] = my_team_picks_df.element.map(full_player_df.set_index('id').points_per_mil)
# my_team_picks_df['position_abv'] = my_team_picks_df.element.map(full_player_df.set_index('id').position_abv)
# my_team_picks_df['team_abv'] = my_team_picks_df.element.map(full_player_df.set_index('id').team_abv)
my_team_picks_df['team_and_position'] = my_team_picks_df['team_abv'].astype(str) + '-' + my_team_picks_df['position_abv'].astype(str)
# my_team_picks_df['prev_season_points_per_gw'] = my_team_picks_df.element.map(full_player_df.set_index('id').prev_season_points_per_gw)
my_team_picks_df['prev_season_points_per_mil'] = my_team_picks_df.prev_season_points_per_gw / my_team_picks_df.cost_in_mil
# my_team_picks_df['next_round_play_chance'] = my_team_picks_df.element.map(full_player_df.set_index('id').next_round_play_chance)

# insert sale price
# my_team_picks_df['potential_profit'] = my_team_picks_df.purchase_price - my_team_picks_df.purchase_price

# do another merge here

# Pulling through Gameweek multipliers
my_team_picks_df['gw_multi'] = my_team_picks_df.team_and_position.map(gameweek_multiplier_df.set_index('team_and_position').gw_multi)
# my_team_picks_df['nxt_5_gw_multi  '] = my_team_picks_df.team_and_position.map(gameweek_multiplier_df.set_index('team_and_position').nxt_5_gw_multi)
# my_team_picks_df['nxt_5_wk_blended_match_multi'] = my_team_picks_df.team_and_position.map(gameweek_multiplier_df.set_index('team_and_position').nxt_5_wk_blended_match_multi)
my_team_picks_df['nxt_3_wk_blended_match_multi'] = my_team_picks_df.team_and_position.map(gameweek_multiplier_df.set_index('team_and_position').nxt_3_wk_blended_match_multi)

my_team_picks_df['player_score'] = ((my_team_picks_df.points_per_mil + my_team_picks_df.prev_season_points_per_mil) * my_team_picks_df.nxt_3_wk_blended_match_multi) * my_team_picks_df.next_round_play_chance

my_team_picks_df.head(11)
    
# #####################
# ##### csv output ####
# #####################

# ## Specifying the filename and path (Location to export the CSV to)
# my_team_filename = 'FPL_My_Team-'
# datetime = dt.now().strftime("%Y%m%d-%H%M%S")
# my_team_path = 'C:\\Python CSV Output\\'+my_team_filename+datetime+'.csv'

# ## Exporting dataframe as csv
# my_team_picks_df.to_csv(my_team_path, index = False)

In [ ]:
## Creating slim_player_df_v2 ##

# Adding current game week and multiplier to slim_player_df by merging with slim_fix_df
# Then removing uneccessary columns from the new merged dataframe

slim_player_fix_df = slim_player_df.copy()
slim_player_fix_df['team_and_position'] = + slim_player_fix_df['team_abv'] + '-' + slim_player_fix_df['position_abv']
slim_player_fix_df['gw_multi'] = slim_player_fix_df.team_and_position.map(gameweek_multiplier_df.set_index('team_and_position').gw_multi)
slim_player_fix_df['nxt_3_wk_blended_match_multi'] = slim_player_fix_df.team_and_position.map(gameweek_multiplier_df.set_index('team_and_position').nxt_3_wk_blended_match_multi)
slim_player_fix_df['negative_multiplier'] = slim_player_fix_df['nxt_3_wk_blended_match_multi'] < 1

calc_player_score = ((slim_player_fix_df.points_per_mil + slim_player_fix_df.prev_season_points_per_mil) * slim_player_fix_df.nxt_3_wk_blended_match_multi) * slim_player_fix_df.next_round_play_chance

# slim_player_fix_df['player_score'] = slim_player_fix_df['nxt_3_wk_blended_match_multi'].apply(lambda x: 0.0 if x < 1 else calc_player_score)

slim_player_fix_df.loc[slim_player_fix_df['negative_multiplier'] == False, 'player_Score'] = calc_player_score  
slim_player_fix_df.loc[slim_player_fix_df['negative_multiplier'] == True,  'player_Score'] = 0.0


# slim_player_fix_df['player_score'] = ((slim_player_fix_df.points_per_mil + slim_player_fix_df.prev_season_points_per_mil) * slim_player_fix_df.nxt_3_wk_blended_match_multi) * slim_player_fix_df.next_round_play_chance

slim_player_fix_df.head(20)


# slim_player_fix_df = slim_player_df_v2.sort_values(by=['player_score'], ascending=False).reset_index(drop = True)

# ## Specifying the filename and path (Location to export the CSV to)
# slim_player_fix_df_filename = 'FPL_slim_player_fix_df-'
# datetime = dt.now().strftime("%Y%m%d-%H%M%S")
# slim_player_fix_df_path = 'C:\\Python CSV Output\\'+slim_player_fix_df_filename+datetime+'.csv'

# # ## Exporting dataframe as csv
# slim_player_fix_df.to_csv(slim_player_fix_df_path, index = False)



### Random Bits / In Development

In [ ]:
##############################################
####### Show Datatypes Of Dataframe ##########
##############################################

# dataTypeSeries = <insert dataframe name>.dtypes
# print('Data type of each column of Dataframe :')
# print(dataTypeSeries)

In [ ]:
###########################################
### print column names from a dataframe ###
###########################################

# for col in club_df.columns: 
#     print(col) 

In [ ]:
## LOGIC TO MAKE A PIVOT TABLE FOR AVERAGE COST PER POSITION (ONLY PLAYERS WHO HAVE PLAYED) AND CONVERT TO A DATAFRAME ##
# The plan was to blend this with player value (points per mil) 
# but I realised this would be detrimental to those players who are less than the average value
# so I shelved this logic

# There might be something in belnd if player cost more than average else just use the points per mil, 
# so that more expensive players are not penalised as much for their higher price

# Without logic like this the very top expensive players like Salah would not have made it into my 2020 team 
# and therefore not banked 40 points as my captain in Gameweek 1!

# Adding Average Cost of position and points per average mil cost of position:

    # A cut of the slim player table with only players who have played for more than 0 minutes
player_has_played_df = full_player_df[full_player_df.minutes != 0]
    # Creating a pivot table to calculate the average cost for the position (only using players who have played this season)
average_cost_pivot = player_has_played_df.pivot_table(index=['position_abv'],
                                                      values=['cost_in_mil'], 
                                                      aggfunc='mean')
    # Then converting to a dataframe and renaming 
average_cost_df = pd.DataFrame(average_cost_pivot.to_records()).rename(columns={"cost_in_mil": "avg_position_cost"})

In [ ]:
##############################################
# Player History - To calculate games played #
##############################################

# Creating an empty list and dictionary
row_list = []
dict1 = {}

# Creating a range of all IDs in 'full_player_df'
id_range = range(1,len(full_player_df)+1)

# For loop to add each individual player json to the row_list 
for i in id_range:
    i_url = 'https://fantasy.premierleague.com/api/element-summary/'+str(i)+'/'
    i_req = requests.get(i_url)
    i_json = i_req.json()
    dict1 = {}
    dict1.update(i_json)
    row_list.append(dict1)

# Converting the row_list into a dataframe   
individual_player_df = pd.DataFrame(row_list)

# Creating a dataframe of just the History part of the above dataframe
# 'Element maps' to id in full_player_df 
player_history_df = pd.DataFrame(individual_player_df['history'])
print(player_history_df)
 
# Need to convert this into a proper dataframe somehow!!!    